In [1]:
import warnings
warnings.catch_warnings() # Dear god qiskit does not play nicely with modern numpy
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")


import random
import math
import numpy as np

import copy

import sys, os, time

# If this starts throwing errors then get the version of qinfer from github
# The version on pip may be severely out of date
from qinfer import LiuWestResampler
from qinfer import utils 

from qiskit import IBMQ
from qiskit import transpile, QuantumRegister, assemble
from qiskit import QuantumCircuit, execute, Aer, QuantumCircuit
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
import qiskit.ignis.verification.randomized_benchmarking as rb
import matplotlib.lines as mlines

#IBMQ.load_account()


Failed to import duecredit due to No module named 'duecredit'
/home/alan/aur/miniconda3/lib/python3.8/site-packages/IPython/parallel.py:12: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  warn("The `IPython.parallel` package has been deprecated since IPython 4.0. "
/home/alan/aur/miniconda3/lib/python3.8/site-packages/qinfer/parallel.py:61: UserWarning: Could not import IPython parallel. Parallelization support will be disabled.
  warnings.warn(


In [2]:
import matplotlib.pyplot as plt
import seaborn as sbs

import smc_linear as smc
Distribution = smc.Distribution

sbs.set(style="darkgrid")

%matplotlib inline

In [3]:
def design_circuit(n_qubits, inv_arr, circuit=None):
    
    if circuit is None:
        circuit = QuantumCircuit(n_qubits, n_qubits)
    
    for i, element in enumerate(inv_arr):
        if element == 1:
            circuit.x(i)
    
    circuit.measure(list(range(n_qubits)), list(range(n_qubits)))
    return circuit

In [4]:
def bv_circuit(bv_string, n_qubits):
        bv_circuit = QuantumCircuit(n_qubits, n_qubits - 1)
        
        for i in range(n_qubits):
            bv_circuit.h(i)
            
        bv_circuit.z(n_qubits - 1)
        
        bv_circuit.barrier()
        
        for i in range(n_qubits -1):
            if bv_string[i] == '1':
                bv_circuit.cx(i, n_qubits - 1)
        
        
        bv_circuit.barrier()
        
        for i in range(n_qubits - 1):
            bv_circuit.h(i)
        
        return bv_circuit

In [5]:
def gen_error_probs(error_arr_c, error_arr_u, error_arr_d, n_qubits = 4, s_penalty=0.3):

    probs = [[0] * (2 ** n_qubits) for _ in range(2 ** n_qubits)]
    
    if len(error_arr_c) != n_qubits + 1:
        raise Exception("Incorrect Error Array")
    
    if len(error_arr_u) != n_qubits + 1:
        raise Exception("Incorrect Error Array")
        
        
    if len(error_arr_d) != n_qubits + 1:
        raise Exception("Incorrect Error Array")
    
    for row in range(2 ** n_qubits):
        row_str = bin(row)[2:].zfill(n_qubits)


        for col in range(2 ** n_qubits):
            col_str = bin(col)[2:].zfill(n_qubits)

            diff_str = [i - j for i, j in zip(list(map(int, row_str)), list(map(int, col_str)))]   
            
            #probs[row][col] -= s_penalty * sum(1 if i == 1 else 0 for i in row_str)
            
            probs[row][col] += error_arr_u[sum(1 if i == -1 else 0 for i in diff_str)]
            probs[row][col] += error_arr_d[sum(1 if i == 1 else 0 for i in diff_str)]
            probs[row][col] += error_arr_c[n_qubits - sum(1 if i == 0 else 0 for i in diff_str)]
            
            probs[row][col] = max(0, probs[row][col])
            
    #Normalise rows, we can then do arbitrary scaling factors in the error arr
    for row, _ in enumerate(probs):
        np_row = np.array(probs[row])
        if sum(np_row) > 0:
            np_row = np_row / sum(np_row) 
        probs[row] = list(np_row)
    
    return probs


In [6]:
def noisy_measure(counts, probs=np.array(gen_error_probs(
        [0,3,3,3,1000], # Const
        [0,1,2,3,4], # 0 -> 1
        [0,9,16,25,36] # 1 -> 0
        )),
        n_qubits=4):
    
    vec = np.zeros((2 ** n_qubits, 1))
    for i in range(2 ** n_qubits):
        try:
            vec[i][0] = counts[str(bin(i)[2:].zfill(n_qubits))]
        except:
            pass
    err_counts = list(map(round, list((probs @ vec).flatten())))    
    counts_final = {}
    for i in range(2 ** n_qubits):
        i_str = bin(i)[2:].zfill(n_qubits)
        counts_final[i_str] = err_counts[i] 
    return counts_final

In [7]:
def measurement_error(counts, n_qubits=4, probs = gen_error_probs(
        [100,10,40,1,1], # Const - Controls correlation of error weights
        [0, 4,3,3,3], # 1 -> 0 - Controls error biases
        [0,-5,-5,-5,-5] # 0 -> 1 - Controls error biases
        )):

    counts_final = noisy_measure(counts, n_qubits=n_qubits, probs=probs)
    return counts_final

In [8]:
def sample_distribution(population, n_shots):
    # There are much more efficient ways to do this
    list_split = []
    for element in population:
        list_split += [element] * population[element]
        
    list_pop = [random.choice(list_split) for _ in range(n_shots)]
    
    new_population = {}
    for element in population:
        new_population[element] = list_pop.count(element)

    return new_population

In [9]:
#provider = IBMQ.get_provider(group='open', project='main')
#backend = provider.get_backend('ibmq_quito') # ibmq_vigo
backend = Aer.get_backend("qasm_simulator")

def smc_runner(circuit, probs = gen_error_probs(
        [100,10,40,1,1], # Const - Controls correlation of error weights
        [0, 4,3,3,3], # 1 -> 0 - Controls error biases
        [0,-5,-5,-5,-5] # 0 -> 1 - Controls error biases
        ), 
        n_shots=1,
    n_measurements = 100,
    n_experiments = 1,
    n_points = 4000):

    results = []

    for i in range(n_experiments):    

        result_data = {}

        result_data['circuit'] = circuit
        result_data['risk'] = []
        result_data['mean'] = []
        result_data['len'] = len(circuit)

        dist = Distribution(n_points=n_points, n_qubits=n_qubits)


        for _ in range(n_measurements):

            inversion_arr = dist.next_experiment()       

            tmp_circuit = copy.deepcopy(circuit)
            tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)


            job = execute(tmp_circuit, backend, shots=1000)

            result = job.result()

            noisy_measurement = measurement_error(result.get_counts(circuit), n_qubits=n_qubits, probs=probs)

            noisy_counts = sample_distribution(noisy_measurement, n_shots)
            #print(noisy_counts)
            outcome = list(map(int, list(list(noisy_counts)[0])))     

            #print("Measurements: {} Outcome: {}".format(inversion_arr, outcome))
            dist.measure(outcome, inversion_arr)
        
        dist.curr_best_circuit()
        mean = dist.calc_bayes_mean()
        inversion_arr = [int(b > a) for a, b in zip(mean[:-1:2], mean[1::2])]
        tmp_circuit = copy.deepcopy(circuit)
        tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)
        
        job = execute(tmp_circuit, backend, shots=1000)

        result = job.result()

        noisy_measurement = measurement_error(result.get_counts(circuit), n_qubits=n_qubits, probs=probs)
        
    return noisy_measurement

In [10]:
# SIM for even numbers of measured qubits 
def sim(circuit, probs, n_shots=1000):
    n_qubits = 4

    sim_strs = [
        [0] * n_qubits, 
        [1] * n_qubits,
        [0, 1] * (n_qubits // 2), 
        [1,0] * (n_qubits // 2)
    ]

    sim_results = {}
    for inversion_arr in sim_strs:

        tmp_circuit = copy.deepcopy(circuit)
        tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)


        job = execute(tmp_circuit, backend, shots=n_shots)

        result = job.result()

        noisy_measurement = measurement_error(result.get_counts(circuit), n_qubits=n_qubits, probs=probs)

        noisy_counts = sample_distribution(noisy_measurement, n_shots)

        for count in noisy_counts:
            count_arr = ''.join(map(str, [i ^ j for i, j in zip(inversion_arr, map(int, list(count[::-1])))]))

            if count_arr in sim_results:
                sim_results[count_arr] += noisy_counts[count]
            else:
                sim_results[count_arr] = noisy_counts[count]



    return sim_results


In [11]:
# SIM for even numbers of measured qubits 
n_qubits = 4

def aim(circuit, probs, n_shots=1000, confirmation_shots=1000):

    aim_strs = [[0] * n_qubits for _ in range(n_qubits)]

    for i in range(len(aim_strs)):
        aim_strs[i][i] = 1


    aim_results = {}
    for inversion_arr in aim_strs:

        tmp_circuit = copy.deepcopy(circuit)
        tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)


        job = execute(tmp_circuit, backend, shots=n_shots)

        result = job.result()

        noisy_measurement = measurement_error(result.get_counts(circuit), n_qubits=n_qubits, probs=probs)

        noisy_counts = sample_distribution(noisy_measurement, n_shots)

        aim_result = {}
        for count in noisy_counts:
            count_arr = ''.join(map(str, [i ^ j for i, j in zip(inversion_arr, map(int, list(count[::-1])))]))

            if count_arr in aim_result:
                aim_result[count_arr] += noisy_counts[count]
            else:
                aim_result[count_arr] = noisy_counts[count]

        aim_results[''.join(map(str, inversion_arr))] = aim_result
    
    # Find max aim
    max_inv_arr = max([(max([b for b in aim_results[i].values()]), i) for i in  aim_results])[1]

    # Run confirmation shots for statistics
    tmp_circuit = copy.deepcopy(circuit)
    tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)

    job = execute(tmp_circuit, backend, shots=confirmation_shots)

    result = job.result()

    noisy_measurement = measurement_error(result.get_counts(circuit), n_qubits=n_qubits, probs=probs)

    noisy_counts = sample_distribution(noisy_measurement, confirmation_shots)
    
    return noisy_counts
        

In [12]:
def ibmq_filter(circuit, probs, n_shots=1000, confirmation_shots=1000):


    qr = QuantumRegister(4)
    meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')

    t_qc = transpile(meas_calibs, backend)
    qobj = assemble(t_qc, shots=n_shots)
    cal_results = backend.run(qobj, shots=n_shots).result()
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')

    job = execute(circuit, backend, shots=confirmation_shots)
    result = job.result()
    noisy_measurement = measurement_error(result.get_counts(circuit), n_qubits=n_qubits, probs=probs)
    noisy_counts = sample_distribution(noisy_measurement, confirmation_shots)

    qiskit_results = meas_fitter.filter.apply(noisy_counts)
    return qiskit_results

In [21]:
for line in probs:
    for val in line:
        print("%.2f " % val, end='')
    print()

0.32 0.13 0.13 0.02 0.13 0.02 0.02 0.02 0.13 0.02 0.02 0.02 0.02 0.02 0.02 0.02 
0.00 0.37 0.04 0.15 0.04 0.15 0.00 0.02 0.04 0.15 0.00 0.02 0.00 0.02 0.00 0.02 
0.00 0.04 0.37 0.15 0.04 0.00 0.15 0.02 0.04 0.00 0.15 0.02 0.00 0.00 0.02 0.02 
0.00 0.00 0.00 0.41 0.04 0.04 0.04 0.16 0.04 0.04 0.04 0.16 0.00 0.00 0.00 0.02 
0.00 0.04 0.04 0.00 0.37 0.15 0.15 0.02 0.04 0.00 0.00 0.00 0.15 0.02 0.02 0.02 
0.00 0.00 0.04 0.04 0.00 0.41 0.04 0.16 0.04 0.04 0.00 0.00 0.04 0.16 0.00 0.02 
0.00 0.04 0.00 0.04 0.00 0.04 0.41 0.16 0.04 0.00 0.04 0.00 0.04 0.00 0.16 0.02 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.48 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.19 
0.00 0.04 0.04 0.00 0.04 0.00 0.00 0.00 0.37 0.15 0.15 0.02 0.15 0.02 0.02 0.02 
0.00 0.00 0.04 0.04 0.04 0.04 0.00 0.00 0.00 0.41 0.04 0.16 0.04 0.16 0.00 0.02 
0.00 0.04 0.00 0.04 0.04 0.00 0.04 0.00 0.00 0.04 0.41 0.16 0.04 0.00 0.16 0.02 
0.00 0.00 0.00 0.00 0.05 0.05 0.05 0.05 0.00 0.00 0.00 0.48 0.05 0.05 0.05 0.19 
0.00 0.04 0.04 0.00 0.00 0.0

In [ ]:
n_qubits = 4
n_shots=40

probs = gen_error_probs(
        [100,0,10,0,0], # Const - Controls correlation of error weights
        [0, 5, 30, 5, 5], # 1 -> 0 - Controls error biases
        [0,-20,-20,-20,-20] # 0 -> 1 - Controls error biases
        )

aim_res = []
sim_res = []
smc_res = []
ibmq_res = []
for i in range(2 ** n_qubits):
    bv_string = bin(i)[2:].zfill(n_qubits)
    
    circuit = bv_circuit(bv_string, n_qubits + 1)
    
    aim_res.append(aim(circuit, probs=probs, n_shots=1000))
    sim_res.append(sim(circuit, probs=probs, n_shots=1000))
    smc_res.append(smc_runner(circuit, probs=probs, n_measurements=n_shots))
    ibmq_res.append(ibmq_filter(circuit, probs, n_shots=n_shots))
    

qc2_sim_probs = [max(r.values()) / sum(list(r.values())) for r in sim_res]
qc2_aim_probs = [max(r.values()) / sum(list(r.values())) for r in aim_res]
qc2_smc_probs = [max(r.values()) / sum(list(r.values())) for r in smc_res]
qc2_ibmq_probs = [max(r.values()) / sum(list(r.values())) for r in ibmq_res]

In [ ]:
plt.title('BV - 4, 2 Qubit Correlated Measurement Errors')
plt.plot(qc2_sim_probs, label='SIM 4000')
plt.plot(qc2_aim_probs, label='AIM 4000')
plt.plot(qc2_smc_probs, label='SMC 40')
plt.plot(qc2_ibmq_probs, linestyle='--', label='Qiskit Filter 16000')
plt.legend()
plt.xlabel('Success Probability')
plt.ylabel('BV State')

In [17]:
n_qubits = 4
n_shots=40

probs = gen_error_probs(
        [100,10,0,0,0], # Const - Controls correlation of error weights
        [0, 30, 5, 5, 5], # 1 -> 0 - Controls error biases
        [0,-20,-20,-20,-20] # 0 -> 1 - Controls error biases
        )

aim_res = []
sim_res = []
smc_res = []
ibmq_res = []
for i in range(2 ** n_qubits):
    bv_string = bin(i)[2:].zfill(n_qubits)
    
    circuit = bv_circuit(bv_string, n_qubits + 1)
    
    aim_res.append(aim(circuit, probs=probs, n_shots=1000))
    sim_res.append(sim(circuit, probs=probs, n_shots=1000))
    smc_res.append(smc_runner(circuit, probs=probs, n_measurements=n_shots))
    ibmq_res.append(ibmq_filter(circuit, probs, n_shots=n_shots))
    

qc1_sim_probs = [max(r.values()) / sum(list(r.values())) for r in sim_res]
qc1_aim_probs = [max(r.values()) / sum(list(r.values())) for r in aim_res]
qc1_smc_probs = [max(r.values()) / sum(list(r.values())) for r in smc_res]
qc1_ibmq_probs = [max(r.values()) / sum(list(r.values())) for r in ibmq_res]

In [ ]:
plt.title('BV - 4, Biased IID Measurement Errors')
plt.plot(qc1_sim_probs, label='AIM 4000')
plt.plot(qc1_aim_probs, label='SIM 4000')
plt.plot(qc1_smc_probs, label='SMC 40')
plt.plot(qc1_ibmq_probs, linestyle='--', label='Qiskit Filter 16000')
plt.legend()
plt.xlabel('Success Probability')
plt.ylabel('BV State')

In [ ]:
n_qubits = 4
n_shots=40

probs = gen_error_probs(
        [500,50,5,1,0.5], # Const - Controls correlation of error weights
        [0, 30, 10, 10, 10], # 1 -> 0 - Controls error biases
        [0,-5,-5,-5,-5] # 0 -> 1 - Controls error biases
        )

aim_res = []
sim_res = []
smc_res = []
ibmq_res = []
for i in range(2 ** n_qubits):
    bv_string = bin(i)[2:].zfill(n_qubits)
    
    circuit = bv_circuit(bv_string, n_qubits + 1)
    
    aim_res.append(aim(circuit, probs=probs, n_shots=1000))
    sim_res.append(sim(circuit, probs=probs, n_shots=1000))
    smc_res.append(smc_runner(circuit, probs=probs, n_measurements=n_shots))
    ibmq_res.append(ibmq_filter(circuit, probs, n_shots=n_shots))
    

qcr_sim_probs = [max(r.values()) / sum(list(r.values())) for r in sim_res]
qcr_aim_probs = [max(r.values()) / sum(list(r.values())) for r in aim_res]
qcr_smc_probs = [max(r.values()) / sum(list(r.values())) for r in smc_res]
qcr_ibmq_probs = [max(r.values()) / sum(list(r.values())) for r in ibmq_res]

In [ ]:
plt.title('BV - 4, Biased IID Measurement Errors')
plt.plot(qcr_sim_probs, label='AIM 4000')
plt.plot(qcr_aim_probs, label='SIM 4000')
plt.plot(qcr_smc_probs, label='SMC 40')
plt.plot(qcr_ibmq_probs, linestyle='--', label='Qiskit Filter 16000')
plt.legend()
plt.xlabel('Success Probability')
plt.ylabel('BV State')